In [3]:
!nvidia-smi

Sat Jan 27 11:32:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8               6W /  80W |    346MiB /  6144MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import torch
from torch import nn

torch.device("cpu"), torch.device("cuda"), torch.device("cuda:1")

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [7]:
torch.cuda.device_count()

1

In [11]:
def try_gpu(i=0):  # @save
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f"cuda:{i}")
    return torch.device("cpu")


def try_all_gpus():  #@save
    devices = [torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device("cpu")]


try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [12]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [13]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [14]:
net = nn.Sequential(nn.Linear(3, 1))
net=net.to(device=try_gpu())

In [15]:
# Z = X.cuda(1)
# X + Y # does't work
# Y + Z
net(X)

tensor([[-0.2488],
        [-0.2488]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)

In [22]:
# the difference in speed between cpu and gpu
import time

size = 500

start_time = time.time()
for i in range(100):
    A = torch.ones(size, size)
    B = torch.ones(size, size)
    C = torch.matmul(A, B)
end_time = time.time()
time1 = end_time - start_time

start_time = time.time()
for i in range(100):
    A = torch.ones(size, size, device=try_gpu())
    B = torch.ones(size, size, device=try_gpu())
    C = torch.matmul(A, B)
end_time = time.time()
time2 = end_time - start_time

print("time in cpu: ", time1)
print("time in gpu: ", time2)

time in cpu:  0.16032195091247559
time in gpu:  0.005866050720214844
